Create IGC
=======

This creates a ISS orbit, and a pushbroom camera. The camera is really different than the real camera (which is a push whisk camera). But as first order this should be vaguely realistic.

In [1]:
from geocal import *

Center times for each of the passes. See the wiki for details on how these were determined

In [2]:
pass_time = [Time.parse_time("2015-01-14T12:00:24.995464Z"),
            Time.parse_time("2015-03-11T13:27:08.758614Z"),
            Time.parse_time("2015-03-02T16:34:33.146760Z"),
            Time.parse_time("2015-04-16T22:13:14.347679Z"),
            Time.parse_time("2015-01-24T14:43:18.819553Z")]

Get the ASTER mosaic we are working with for all the bands. This comes from the wiki. Data originally comes from /raid11/astermos, but is compressed there.

In [7]:
def fname(band):
    return "/data/smyth/AsterMosiac/calnorm_b%d.img" % band
f = [VicarLiteRasterImage(fname(aster_band)) for aster_band in [10, 11, 12, 14, 14, 4]]

In [9]:
orb = SpiceOrbit(SpiceOrbit.ISS_ID, "iss_spice/iss_2015.bsp")
print(Geodetic(orb.position_cf(pass_time[0])))

Geodetic: (39.5 deg, -122.07 deg, 414397 m)


Focal length and ccd pixel size comes from Eugene's SDS data bible. The scaling of the CCD size is empirical to give the right resolution on the surface. These are pretty hoaky, we really just want something vaguely right since our camera model is pretty different from a pushbroom. But this gives a place to start.

In [11]:
frame_to_sc = quat_rot("ZYX", 0, 0, 0)
cam = QuaternionCamera(frame_to_sc, 1, 5400, 40e-3 * 1.8, 40e-3 * 2, 425, FrameCoordinate(1.0 / 2, 5400.0 / 2))
print(cam)

QuaternionCamera:
   Number line:     1
   Number sample:   5400
   Focal length:    425 mm
   Line pitch:      0.072 mm
   Sample pitch:    0.08 mm
   Principal point: Frame Coordinate: (0.5, 2700)
   Frame convention: LINE_IS_X
   Frame to spacecraft: (1,0,0,0)



The time table data comes from Eugene's SDS data bible file (ECOSTRESS_SDS_Data_Bible.xls in ecostress-sds git repository). The real camera is a bit complicated, but we collect about 241 samples of data (in along track direction) every 1.181 seconds. For a pushbroom, we can divide this up evenly as an approximation. But then there is an averaging step (which I don't know the details of) that combines 2 pixels. So we have the factor of 2 given. Scene has 5400 pixels, which is where time calcuation comes from.

In [13]:
# False here says it ok for SrtmDem to not have tile. This gives support
# for data that is over the ocean.
dem = SrtmDem("",False)
band = 0
tspace = 1.181 / 241 * 2
toff = 5400 * tspace / 2
tt = ConstantSpacingTimeTable(pass_time[0] - toff + tspace, pass_time[0] + toff, tspace)
ipi = Ipi(orb, cam, band, tt.min_time, tt.max_time, tt)
igc = IpiImageGroundConnection(ipi, dem, None)

In [15]:
print(distance(igc.ground_coordinate(ImageCoordinate(1, 5400 / 2)), 
               igc.ground_coordinate(ImageCoordinate(1, 1 + 5400 / 2))))
print(distance(igc.ground_coordinate(ImageCoordinate(1, 5400 / 2)), 
               igc.ground_coordinate(ImageCoordinate(2, 5400 / 2))))

77.96361047421502
67.75143691654624


In [19]:
print(igc.footprint_resolution(1, int(5400 / 2)))

(70.16135516404367, 77.95709168959802)


In [11]:
write_shelve("igc_test.xml", igc)
write_shelve("aster_rad.xml", f)
# We may want to have IgcSimulated not read everything into memory. But start with this
write_shelve("test_data.db:igc_simulated", IgcSimulated(igc, f, -1, False))
# Then do write_image --verbose --number-process=20 --process-number-line=100 --process-number-sample=100 test_data.db:igc_simulated l1b1_sim.img 

In [12]:
print f.coordinate(igc.ground_coordinate(ImageCoordinate(0,0)))
print f.coordinate(igc.ground_coordinate(ImageCoordinate(0,igc.number_sample - 1)))
print f.coordinate(igc.ground_coordinate(ImageCoordinate(igc.number_line - 1, 0)))
print f.coordinate(igc.ground_coordinate(ImageCoordinate(igc.number_line - 1, igc.number_sample - 1)))

(249.462, 18524.1)
(22085, -1894.57)
(15371.8, 42765.2)
(36555.8, 21889.5)


In [13]:
write_shelve("orbit.xml", orb)
write_shelve("camera.xml", cam)
write_shelve("time_table.xml", tt)